# Time Series Data Preparation

Walmart Sales dataset, collected from [Kaggle](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data), contains historical sales data for 45 Walmart stores in the United State from 2/5/2010 to 11/1/2012.

In [1]:
import zipfile
import datetime
import pandas as pd
import numpy as np

# load sales data
sales = pd.read_csv('data/walmart_sales.zip',compression='zip')
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [2]:
sales.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


## Prepare Time Series Dataset

In [3]:
# keep only Date and Weekly_Sales columns
weekly_sales = sales[['Date', 'Weekly_Sales']].copy()

# convert data type for Date from object to datetime
weekly_sales['Date'] = pd.to_datetime(weekly_sales['Date'])

In [4]:
weekly_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Date          421570 non-null  datetime64[ns]
 1   Weekly_Sales  421570 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 6.4 MB


In [5]:
weekly_sales.head()

,Date,Weekly_Sales
0,2010-02-05,24924.50
1,2010-02-12,46039.49
2,2010-02-19,41595.55
3,2010-02-26,19403.54
4,2010-03-05,21827.90


### Aggregate Sales Data

In [6]:
# compute total weekly sales across all stores
weekly_sales = weekly_sales.groupby(['Date']).agg(sum).reset_index()
weekly_sales

,Date,Weekly_Sales
0,2010-02-05,49750740.50
1,2010-02-12,48336677.63
2,2010-02-19,48276993.78
3,2010-02-26,43968571.13
4,2010-03-05,46871470.30
...,...,...
138,2012-09-28,43734899.40
139,2012-10-05,47566639.31
140,2012-10-12,46128514.25
141,2012-10-19,45122410.57


### Impute Missing Data

In [7]:
# extract week number of year from Date
weekly_sales['Week'] = weekly_sales['Date'].map(lambda x: datetime.date(x.year, x.month, x.day).isocalendar().week)

In [8]:
# list sales data sorted by Date
weekly_sales.sort_values(by=['Date'])

# found missing sales values for:
#  - the first 4 weeks of 2010
#  - the last 9 weeks of 2012

,Date,Weekly_Sales,Week
0,2010-02-05,49750740.50,5
1,2010-02-12,48336677.63,6
2,2010-02-19,48276993.78,7
3,2010-02-26,43968571.13,8
4,2010-03-05,46871470.30,9
...,...,...,...
138,2012-09-28,43734899.40,39
139,2012-10-05,47566639.31,40
140,2012-10-12,46128514.25,41
141,2012-10-19,45122410.57,42


In [9]:
def fill_missing_weeks(data: pd.DataFrame, 
                       n_weeks: int, 
                       target='Weekly_Sales', 
                       method='forward') -> pd.DataFrame:
    '''
    Fill in sales data for the missing weeks using the averages of the corresponding weeks from other years.
    Return a data frame of missing sales data.
    
    Parms:
      - data: a Pandas dataframe containing sales data
      - n_weeks: number of missing weeks
      - target: label of target variable
      - method: fill missing values for the following weeks (forward) or the previous weeks (backward)
    '''
    
    missing_weeks = []
    start_date = data.Date.iloc[-1] if method == 'forward' else data.Date.iloc[0]
    
    for i in range(0, n_weeks):
        # get the base date for computation
        current_date = start_date if len(missing_weeks) == 0 else missing_weeks[-1]
        
        if method == 'forward':   # get date the next following week
            missing_weeks.append(current_date + datetime.timedelta(days=7))
        else:   # get date of the previous week
            missing_weeks.append(current_date - datetime.timedelta(days=7))
    
    # create a data frame containing the date of missing weeks
    missing_data = pd.DataFrame({'Date': missing_weeks})
    missing_data['Week'] = missing_data['Date'].map(lambda x: datetime.date(x.year, x.month, x.day).isocalendar().week)
    
    # fill in sales data for the missing weeks
    filters = data.Week >= missing_data.Week.iloc[0] if method == 'forward' else data.Week <= missing_data.Week.max()
    missing_data[target] = data[filters].groupby(['Week'])[target].mean().values
        
    return missing_data

In [10]:
# fill in sales values for the missing weeks using the mean of the corresponding weeks from other years
weekly_sales = pd.concat([weekly_sales,
                          fill_missing_weeks(weekly_sales, 4, method='backward'),
                          fill_missing_weeks(weekly_sales, 9)], 
                         ignore_index=True).sort_values(by=['Date']).reset_index(drop=True)
weekly_sales

,Date,Weekly_Sales,Week
0,2010-01-08,3.971741e+07,1
1,2010-01-15,4.136782e+07,2
2,2010-01-22,4.134838e+07,3
3,2010-01-29,4.386560e+07,4
4,2010-02-05,4.975074e+07,5
...,...,...,...
151,2012-11-30,4.964979e+07,48
152,2012-12-07,5.561396e+07,49
153,2012-12-14,6.095325e+07,50
154,2012-12-21,7.896483e+07,51


### Transform Sale Values

In [11]:
# convert sales value to million
weekly_sales['Weekly Sales (Million)'] = weekly_sales['Weekly_Sales'] / 1000000

# transform sales value: log/square
weekly_sales['Log of Weekly Sales (Million)'] = np.log(weekly_sales['Weekly Sales (Million)'])
weekly_sales['Square Root of Weekly Sales (Million)'] = np.sqrt(weekly_sales['Weekly Sales (Million)'])

# drop Weekly_Sales
del weekly_sales['Weekly_Sales']
weekly_sales

,Date,Week,Weekly Sales (Million),Log of Weekly Sales (Million),Square Root of Weekly Sales (Million)
0,2010-01-08,1,39.717414,3.681790,6.302175
1,2010-01-15,2,41.367822,3.722503,6.431782
2,2010-01-22,3,41.348378,3.722033,6.430270
3,2010-01-29,4,43.865605,3.781131,6.623111
4,2010-02-05,5,49.750740,3.907025,7.053420
...,...,...,...,...,...
151,2012-11-30,48,49.649792,3.904994,7.046261
152,2012-12-07,49,55.613959,4.018434,7.457477
153,2012-12-14,50,60.953248,4.110107,7.807256
154,2012-12-21,51,78.964828,4.369003,8.886216


### Assign Holiday Name

In [12]:
# create a data frame of holidays
holidays = pd.DataFrame(dict({'Super Bowl': ['2010-02-12', '2011-02-11', '2012-02-10', '2012-02-08'],
                              'Labor Day': ['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06'],
                              'Thanksgiving': ['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29'],
                              'Christmas': ['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27']}))

# change data type for each column to datetime
for col_name in holidays.columns:
    holidays[col_name] = pd.to_datetime(holidays[col_name])

def assign_holiday(date):
    '''
    Return holiday's name if holiday falls within a given week; otherwise, return empty string.
    '''
    
    global holidays
    
    # assign holiday name to specific date
    for col_name in holidays.columns:
        # check if holiday is in a given week
        isHoliday = holidays[col_name] == date
        
        # return holiday name if holiday is within the given week
        if len(holidays[isHoliday]) > 0:
            return col_name
    
    # return empty string if holiday is not in the given week
    return ''

In [13]:
# get holiday's name if holiday falls within any specific weeks
weekly_sales['Holiday'] = weekly_sales['Date'].apply(assign_holiday)

In [14]:
weekly_sales[weekly_sales.Holiday != '']

,Date,Week,Weekly Sales (Million),Log of Weekly Sales (Million),Square Root of Weekly Sales (Million),Holiday
5,2010-02-12,6,48.336678,3.878191,6.952458,Super Bowl
35,2010-09-10,36,45.634398,3.820662,6.755324,Labor Day
46,2010-11-26,47,65.821003,4.186939,8.113014,Thanksgiving
51,2010-12-31,52,40.432519,3.699634,6.358657,Christmas
57,2011-02-11,6,47.336193,3.857275,6.880130,Super Bowl
87,2011-09-09,36,46.763228,3.845097,6.838364,Labor Day
98,2011-11-25,47,66.593605,4.198609,8.160491,Thanksgiving
103,2011-12-30,52,46.042461,3.829564,6.785460,Christmas
109,2012-02-10,6,50.009408,3.912211,7.071733,Super Bowl
139,2012-09-07,36,48.330059,3.878054,6.951982,Labor Day


### Set Train/Test Dataset

In [15]:
# set time series index
weekly_sales['t'] = weekly_sales.index

# set train/test dataset
weekly_sales['Dataset'] = weekly_sales['Date'].map(lambda x: 
          'Test' if datetime.date(x.year, x.month, x.day).isocalendar().year > 2011 else 'Train')


In [16]:
weekly_sales[weekly_sales.Dataset == 'Train']

,Date,Week,Weekly Sales (Million),Log of Weekly Sales (Million),Square Root of Weekly Sales (Million),Holiday,t,Dataset
0,2010-01-08,1,39.717414,3.681790,6.302175,,0,Train
1,2010-01-15,2,41.367822,3.722503,6.431782,,1,Train
2,2010-01-22,3,41.348378,3.722033,6.430270,,2,Train
3,2010-01-29,4,43.865605,3.781131,6.623111,,3,Train
4,2010-02-05,5,49.750740,3.907025,7.053420,,4,Train
...,...,...,...,...,...,...,...,...
99,2011-12-02,48,49.390556,3.899759,7.027842,,99,Train
100,2011-12-09,49,55.561148,4.017484,7.453935,,100,Train
101,2011-12-16,50,60.085696,4.095772,7.751496,,101,Train
102,2011-12-23,51,76.998241,4.343783,8.774864,,102,Train


In [17]:
weekly_sales[weekly_sales.Dataset == 'Test']

,Date,Week,Weekly Sales (Million),Log of Weekly Sales (Million),Square Root of Weekly Sales (Million),Holiday,t,Dataset
104,2012-01-06,1,44.955422,3.805671,6.704880,,104,Test
105,2012-01-13,2,42.023078,3.738219,6.482521,,105,Test
106,2012-01-20,3,42.080997,3.739596,6.486987,,106,Test
107,2012-01-27,4,39.834975,3.684745,6.311495,,107,Test
108,2012-02-03,5,46.085608,3.830501,6.788638,,108,Test
109,2012-02-10,6,50.009408,3.912211,7.071733,Super Bowl,109,Test
110,2012-02-17,7,50.197057,3.915956,7.084988,,110,Test
111,2012-02-24,8,45.771507,3.823662,6.765464,,111,Test
112,2012-03-02,9,46.861035,3.847187,6.845512,,112,Test
113,2012-03-09,10,47.480454,3.860318,6.890606,,113,Test


## Save Preprocessed Data

In [18]:
# save dataset to csv files
weekly_sales.to_csv('data/ts_dataset.csv', index=False)